In [1]:
# Import necessary libraries
import os
import numpy as np
import cv2
from PIL import Image
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm
import sys

from config import PROCESSED_DATASET_PATH

# Try to import LaMa-specific modules
try:
    print(torch.cuda.is_available())
    from iopaint.model import LaMa
    from iopaint.schema import InpaintRequest, LDMSampler

    config = InpaintRequest(
    mask_strategy="custom",
    ldm_steps=50,  # Adjust as needed
    ldm_sampler=LDMSampler.ddim,  # or other sampler options
    hd_strategy="Original",  # or "Resize", "Crop"
    hd_strategy_crop_margin=32,
    hd_strategy_crop_trigger_size=512,
    hd_strategy_resize_limit=512
    )
    
    # Check if CUDA is available
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")
    
    # Initialize the LaMa model
    model = LaMa(device=device)
    print("LaMa model loaded successfully")
    
    # Print version information
    import pkg_resources
    iopaint_version = pkg_resources.get_distribution("iopaint").version
    print(f"iopaint version: {iopaint_version}")
    
    # Test with a sample image
    # You can replace this with an actual image path
    sample_image_path = r"C:\Users\rberg\Documents\quick-look\data\filtered_dataset\processed\masked\000000000110.png" #provide the masked image
    sample_mask_path = r"C:\Users\rberg\Documents\quick-look\data\filtered_dataset\processed\masks\000000000110.png"
    sample_output_path = "./test-lama.png"

    
    if os.path.exists(sample_image_path):
        # Load the image
        image = cv2.imread(sample_image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Create a simple mask (white rectangle in the middle)
        mask = cv2.imread(sample_mask_path, cv2.IMREAD_GRAYSCALE)
        height, width = image.shape[:2]
        mask_width, mask_height = width // 3, height // 3
        x1, y1 = width // 3, height // 3
        x2, y2 = 2 * width // 3, 2 * height // 3
        
        
        # Inpaint the image
        result = model(image, mask, config)
        
        # Create a canvas for displaying all images side by side
        height, width = image.shape[:2]
        canvas = np.zeros((height, width * 3, 3), dtype=np.uint8)
        
        # Add original image to the canvas
        canvas[:, 0:width] = image
        
        # Add mask to the canvas (convert grayscale to RGB for display)
        mask_rgb = cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB)
        canvas[:, width:width*2] = mask_rgb
        
        # Add inpainted result to the canvas
        canvas[:, width*2:width*3] = result
        
        # Add titles
        title_height = 30
        title_canvas = np.ones((title_height, width * 3, 3), dtype=np.uint8) * 255
        
        # Add text for titles
        cv2.putText(title_canvas, "Original Image", (width//2 - 70, 20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(title_canvas, "Mask", (width + width//2 - 30, 20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(title_canvas, "Inpainted Result", (width*2 + width//2 - 80, 20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        
        # Combine title and images
        display_image = np.vstack((title_canvas, canvas))
        
        # Save the combined image
        cv2.imwrite(sample_output_path, cv2.cvtColor(display_image, cv2.COLOR_RGB2BGR))
        
        print(f"Image saved successfully to {sample_output_path}")
    else:
        print(f"Sample image not found at {sample_image_path}")
        print("Please update the path to test with your own image")

except ImportError as e:
    print(f"LaMa dependencies not installed: {e}")
    print("Please install iopaint package with: pip install iopaint")
    print(f"Full error: {str(e)}")
    import traceback
    traceback.print_exc()


True


c:\Users\rberg\Anaconda3\envs\seg-env-win-friendly\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-04 21:09:19.112 | INFO     | iopaint.helper:load_jit_model:107 - Loading model from: C:\Users\rberg/.cache\torch\hub\checkpoints\big-lama.pt


Using device: cuda
LaMa model loaded successfully
iopaint version: 1.6.0


C:\Users\rberg\AppData\Local\Temp\ipykernel_22164\4044462445.py:38: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


Image saved successfully to ./test-lama.png


In [5]:
# Import necessary libraries
import os
import numpy as np
import cv2
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm
import sys

from config import PROCESSED_DATASET_PATH

from iopaint.model import LDM
from iopaint.schema import InpaintRequest, LDMSampler

# Try to import LaMa-specific modules
try:
    print(torch.cuda.is_available())
    from iopaint.model import LDM
    from iopaint.schema import InpaintRequest, LDMSampler

    # Initialize LDM model (Stable Diffusion)
    model = LDM(name="sd1.5", device="cpu" if not torch.cuda.is_available() else "cuda")

    
    # Create inpaint request with text prompt
    request = InpaintRequest(
        mask_strategy="custom",
        prompt="chair",
        negative_prompt="blurry, bad quality",
        ldm_sampler=LDMSampler.ddim,  # Choose sampler
        ldm_steps=50,
        hd_strategy="Original"
    )
    
    # Print version information
    import pkg_resources
    iopaint_version = pkg_resources.get_distribution("iopaint").version
    print(f"iopaint version: {iopaint_version}")
    
    # Test with a sample image
    # You can replace this with an actual image path
    sample_image_path = r"C:\Users\rberg\Documents\quick-look\data\filtered_dataset\processed\masked\000000000110.png" #provide the masked image
    sample_mask_path = r"C:\Users\rberg\Documents\quick-look\data\filtered_dataset\processed\masks\000000000110.png"
    sample_output_path = "./test-ldm.png"

    
    if os.path.exists(sample_image_path):
        # Load the image
        image = cv2.imread(sample_image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Create a simple mask (white rectangle in the middle)
        mask = cv2.imread(sample_mask_path, cv2.IMREAD_GRAYSCALE)
        
        # Inpaint the image - directly use numpy array
        result = model(image, mask, request)
        
        # Convert result back to numpy array if needed
        if not isinstance(result, np.ndarray):
            result = np.array(result)
        
        # Swap the R and B channels of the result image
        # In RGB format, channels are ordered as [R, G, B] with indices [0, 1, 2]
        # We'll create a copy to avoid modifying the original array
        result_rb_swapped = result.copy()
        
        # Swap R (index 0) and B (index 2) channels
        result_rb_swapped[:, :, 0] = result[:, :, 2]  # R becomes B
        result_rb_swapped[:, :, 2] = result[:, :, 0]  # B becomes R
        
        # Update the result with the R-B swapped version
        result = result_rb_swapped
        # Create a canvas for displaying all images side by side
        height, width = image.shape[:2]
        canvas = np.zeros((height, width * 3, 3), dtype=np.uint8)
        
        # Add original image to the canvas
        canvas[:, 0:width] = image
        
        # Add mask to the canvas (convert grayscale to RGB for display)
        mask_rgb = cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB)
        canvas[:, width:width*2] = mask_rgb
        
        # Add inpainted result to the canvas
        canvas[:, width*2:width*3] = result
        
        # Add titles
        title_height = 30
        title_canvas = np.ones((title_height, width * 3, 3), dtype=np.uint8) * 255
        
        # Add text for titles
        cv2.putText(title_canvas, "Original Image", (width//2 - 70, 20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(title_canvas, "Mask", (width + width//2 - 30, 20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(title_canvas, "Inpainted Result", (width*2 + width//2 - 80, 20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        
        # Combine title and images
        display_image = np.vstack((title_canvas, canvas))
        
        # Save the combined image
        cv2.imwrite(sample_output_path, cv2.cvtColor(display_image, cv2.COLOR_RGB2BGR))
        
        print(f"Image saved successfully to {sample_output_path}")
    else:
        print(f"Sample image not found at {sample_image_path}")
        print("Please update the path to test with your own image")

except ImportError as e:
    print(f"LaMa dependencies not installed: {e}")
    print("Please install iopaint package with: pip install iopaint")
    print(f"Full error: {str(e)}")
    import traceback
    traceback.print_exc()

class LamaGapFiller:
    """
    Gap filling using LaMa (Large Mask Inpainting) model via iopaint.
    """
    def __init__(self, device=None):
        """
        Initialize the LaMa gap filler.
        
        Args:
            device: Device to run the model on ('cuda' or 'cpu')
        """
        self.device = device if device else ("cuda" if torch.cuda.is_available() else "cpu")
        self._load_model()
    
    def _load_model(self):
        try:
            # Import LaMa-specific modules
            from iopaint.model import LDM
            from iopaint.schema import InpaintRequest, LDMSampler
            
            # Initialize LDM model (Stable Diffusion)
            self.model = LDM(name="sd1.5", device=self.device)
            
            # Create inpaint request with text prompt
            self.request = InpaintRequest(
                mask_strategy="custom",
                prompt="chair",
                negative_prompt="blurry, bad quality",
                ldm_sampler=LDMSampler.ddim,
                ldm_steps=50,
                hd_strategy="Original"
            )
            
            print("LDM model loaded successfully")
        except ImportError as e:
            print(f"iopaint dependencies not installed: {e}")
            self.model = None
    
    def fill_gap(self, image_with_gap, mask=None):
        """
        Fill the gap using LDM inpainting model.
        
        Args:
            image_with_gap: Image with a gap to be filled (numpy array, RGB)
            mask: Binary mask indicating the gap region (1 for gap, 0 for known regions)
            
        Returns:
            Completed image with the gap filled
        """
        if self.model is None:
            print("Model not loaded. Returning original image.")
            return image_with_gap
        
        # Ensure mask is in the correct format
        if mask is None:
            # Try to detect the gap automatically (assuming white/light gray areas)
            gray = cv2.cvtColor(image_with_gap, cv2.COLOR_RGB2GRAY)
            mask = np.zeros_like(gray)
            mask[(gray > 240)] = 255  # Adjust threshold as needed
        else:
            # Convert binary mask (0-1) to grayscale (0-255) if needed
            if mask.max() <= 1:
                mask = (mask * 255).astype(np.uint8)
        
        # Inpaint the image
        result = self.model(image_with_gap, mask, self.request)
        
        # Convert result back to numpy array if needed
        if not isinstance(result, np.ndarray):
            result = np.array(result)
        
        # Swap the R and B channels of the result image
        # In RGB format, channels are ordered as [R, G, B] with indices [0, 1, 2]
        result_rb_swapped = result.copy()
        result_rb_swapped[:, :, 0] = result[:, :, 2]  # R becomes B
        result_rb_swapped[:, :, 2] = result[:, :, 0]  # B becomes R
        
        return result_rb_swapped


2025-05-04 21:21:23.658 | INFO     | iopaint.helper:load_jit_model:107 - Loading model from: C:\Users\rberg/.cache\torch\hub\checkpoints\diffusion.pt


True


2025-05-04 21:21:25.004 | INFO     | iopaint.helper:load_jit_model:107 - Loading model from: C:\Users\rberg/.cache\torch\hub\checkpoints\cond_stage_model_decode.pt
2025-05-04 21:21:25.120 | INFO     | iopaint.helper:load_jit_model:107 - Loading model from: C:\Users\rberg/.cache\torch\hub\checkpoints\cond_stage_model_encode.pt
2025-05-04 21:21:25.392 | INFO     | iopaint.model.ddim_sampler:ddim_sampling:121 - Running DDIM Sampling with 50 timesteps


iopaint version: 1.6.0


DDIM Sampler: 100%|██████████| 50/50 [00:14<00:00,  3.34it/s]


Image saved successfully to ./test-ldm.png


In [6]:
# Import necessary libraries
import os
import numpy as np
import cv2
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm
import sys

from config import PROCESSED_DATASET_PATH

# Try to import iopaint modules
try:
    print(torch.cuda.is_available())
    from iopaint.model import PaintByExample
    
    # Initialize PaintByExample model
    model = PaintByExample(device="cuda" if torch.cuda.is_available() else "cpu")
    
    # Print version information
    import pkg_resources
    iopaint_version = pkg_resources.get_distribution("iopaint").version
    print(f"iopaint version: {iopaint_version}")
    
    # Test with sample images
    sample_image_path = r"C:\Users\rberg\Documents\quick-look\data\filtered_dataset\processed\masked\000000000110.png" # masked image
    sample_mask_path = r"C:\Users\rberg\Documents\quick-look\data\filtered_dataset\processed\masks\000000000110.png"
    # Example image path - this is the reference image that contains what we want to paint
    example_image_path = r"C:\Users\rberg\Documents\quick-look\data\misc_files\chair.jpg"
    sample_output_path = "./test-paintbyexample.png"
    
    if os.path.exists(sample_image_path) and os.path.exists(example_image_path):
        # Load the image
        image = cv2.imread(sample_image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Load the example image
        example_image = cv2.imread(example_image_path)
        example_image = cv2.cvtColor(example_image, cv2.COLOR_BGR2RGB)
        
        # Load the mask
        mask = cv2.imread(sample_mask_path, cv2.IMREAD_GRAYSCALE)
        
        # Inpaint the image using PaintByExample
        result = model(image, mask, example_image=example_image)
        
        # Convert result back to numpy array if needed
        if not isinstance(result, np.ndarray):
            result = np.array(result)
        
        # Swap the R and B channels of the result image
        # In RGB format, channels are ordered as [R, G, B] with indices [0, 1, 2]
        # We'll create a copy to avoid modifying the original array
        result_rb_swapped = result.copy()
        
        # Swap R (index 0) and B (index 2) channels
        result_rb_swapped[:, :, 0] = result[:, :, 2]  # R becomes B
        result_rb_swapped[:, :, 2] = result[:, :, 0]  # B becomes R
        
        # Update the result with the R-B swapped version
        result = result_rb_swapped
        # Create a canvas for displaying all images side by side
        height, width = image.shape[:2]
        canvas = np.zeros((height, width * 4, 3), dtype=np.uint8)
        
        # Add original image to the canvas
        canvas[:, 0:width] = image
        
        # Add mask to the canvas (convert grayscale to RGB for display)
        mask_rgb = cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB)
        canvas[:, width:width*2] = mask_rgb
        
        # Add example image to the canvas
        canvas[:, width*2:width*3] = example_image
        
        # Add inpainted result to the canvas
        canvas[:, width*3:width*4] = result
        
        # Add titles
        title_height = 30
        title_canvas = np.ones((title_height, width * 4, 3), dtype=np.uint8) * 255
        
        # Add text for titles
        cv2.putText(title_canvas, "Original Image", (width//2 - 70, 20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(title_canvas, "Mask", (width + width//2 - 30, 20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(title_canvas, "Example Image", (width*2 + width//2 - 70, 20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(title_canvas, "Inpainted Result", (width*3 + width//2 - 80, 20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        
        # Combine title and images
        display_image = np.vstack((title_canvas, canvas))
        
        # Save the combined image
        cv2.imwrite(sample_output_path, cv2.cvtColor(display_image, cv2.COLOR_RGB2BGR))
        
        print(f"Image saved successfully to {sample_output_path}")
    else:
        print(f"Sample image not found at {sample_image_path} or example image not found at {example_image_path}")
        print("Please update the paths to test with your own images")

except ImportError as e:
    print(f"iopaint dependencies not installed: {e}")
    print("Please install iopaint package with: pip install iopaint")
    print(f"Full error: {str(e)}")
    import traceback
    traceback.print_exc()


True


KeyError: 'model_info'

In [ ]:
# Import necessary libraries
import os
import numpy as np
import cv2
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm
import sys

from config import PROCESSED_DATASET_PATH

# Try to import PaintByExample-specific modules
try:
    print(torch.cuda.is_available())
    from iopaint.model import PaintByExample
    from iopaint.schema import InpaintRequest

    # Initialize PaintByExample model
    device = "cpu" if not torch.cuda.is_available() else "cuda"
    
    # PaintByExample requires a reference image instead of a text prompt
    model = PaintByExample(device=device)
    
    # Print version information
    import pkg_resources
    iopaint_version = pkg_resources.get_distribution("iopaint").version
    print(f"iopaint version: {iopaint_version}")
    
   # Test with sample images
    sample_image_path = r"C:\Users\rberg\Documents\quick-look\data\filtered_dataset\processed\masked\000000000110.png" # masked image
    sample_mask_path = r"C:\Users\rberg\Documents\quick-look\data\filtered_dataset\processed\masks\000000000110.png"
    # Example image path - this is the reference image that contains what we want to paint
    reference_image_path = r"C:\Users\rberg\Documents\quick-look\data\misc_files\chair.jpg"
    sample_output_path = "./test-paintbyexample.png"
    
    # Load the reference image (this will be used as the example)
    if not os.path.exists(reference_image_path):
        print(f"Reference image not found at {reference_image_path}")
        print("Please provide a path to a reference image")
        sys.exit(1)
    
    reference_image = cv2.imread(reference_image_path)
    reference_image = cv2.cvtColor(reference_image, cv2.COLOR_BGR2RGB)
    
    if os.path.exists(sample_image_path):
        # Load the image to be inpainted
        image = cv2.imread(sample_image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Load the mask
        mask = cv2.imread(sample_mask_path, cv2.IMREAD_GRAYSCALE)
        
        # Create inpaint request
        request = InpaintRequest(
            mask_strategy="custom",
            hd_strategy="Original"
        )
        
        # Inpaint the image using the reference image
        # PaintByExample takes: image, mask, reference_image
        result = model.forward(image, mask, reference_image, request)
        
        # Convert result back to numpy array if needed
        if not isinstance(result, np.ndarray):
            result = np.array(result)
        
        # Create a canvas for displaying all images side by side
        height, width = image.shape[:2]
        canvas = np.zeros((height, width * 4, 3), dtype=np.uint8)
        
        # Resize reference image to match height if needed
        if reference_image.shape[0] != height:
            aspect_ratio = reference_image.shape[1] / reference_image.shape[0]
            new_width = int(height * aspect_ratio)
            reference_image_resized = cv2.resize(reference_image, (new_width, height))
        else:
            reference_image_resized = reference_image
        
        # Add original image to the canvas
        canvas[:, 0:width] = image
        
        # Add mask to the canvas (convert grayscale to RGB for display)
        mask_rgb = cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB)
        canvas[:, width:width*2] = mask_rgb
        
        # Add reference image to the canvas
        canvas[:, width*2:width*3] = reference_image_resized[:, :width]  # Crop if needed
        
        # Add inpainted result to the canvas
        canvas[:, width*3:width*4] = result
        
        # Add titles
        title_height = 30
        title_canvas = np.ones((title_height, width * 4, 3), dtype=np.uint8) * 255
        
        # Add text for titles
        cv2.putText(title_canvas, "Original Image", (width//2 - 70, 20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(title_canvas, "Mask", (width + width//2 - 30, 20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(title_canv

In [8]:
# Import necessary libraries
import os
import numpy as np
import cv2
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm
import sys

from config import PROCESSED_DATASET_PATH

# Try to import PaintByExample-specific modules
try:
    print(torch.cuda.is_available())
    from iopaint.model_manager import ModelManager
    from iopaint.schema import InpaintRequest
    from iopaint.model.paint_by_example import PaintByExample


    # Initialize PaintByExample model using ModelManager
    device = "cpu" if not torch.cuda.is_available() else "cuda"
    
    # Create model using ModelManager with PaintByExample model name
    model = ModelManager(
        name=PaintByExample.name,
        device=device,
        disable_nsfw=False,
        disable_nsfw_checker=False
    )
    
    # Print version information
    import pkg_resources
    iopaint_version = pkg_resources.get_distribution("iopaint").version
    print(f"iopaint version: {iopaint_version}")
    
    # Paths to your files
    sample_image_path = r"C:\Users\rberg\Documents\quick-look\data\filtered_dataset\processed\masked\000000000110.png"  # Image to inpaint
    sample_mask_path = r"C:\Users\rberg\Documents\quick-look\data\filtered_dataset\processed\masks\000000000110.png"     # Mask showing area to inpaint
    reference_image_path = r"C:\path\to\reference\image.png"  # Reference image for PaintByExample
    sample_output_path = "./test-paintbyexample.png"
    
    # Load the reference image (this will be used as the example)
    if not os.path.exists(reference_image_path):
        print(f"Reference image not found at {reference_image_path}")
        print("Please provide a path to a reference image")
        sys.exit(1)
    
    reference_image = cv2.imread(reference_image_path)
    reference_image = cv2.cvtColor(reference_image, cv2.COLOR_BGR2RGB)
    
    if os.path.exists(sample_image_path):
        # Load the image to be inpainted
        image = cv2.imread(sample_image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Load the mask
        mask = cv2.imread(sample_mask_path, cv2.IMREAD_GRAYSCALE)
        
        # Create inpaint request
        request = InpaintRequest(
            mask_strategy="custom",
            hd_strategy="Original"
        )
        
        # Inpaint the image using the reference image
        # Use the ModelManager's forward method
        # PaintByExample requires passing the request with the image by example
        # We need to set the reference image in the request
        request.example_image = reference_image
        
        result = model(image=image, mask=mask, request=request)
        
        # Convert result back to numpy array if needed
        if not isinstance(result, np.ndarray):
            result = np.array(result)
        
        # Create a canvas for displaying all images side by side
        height, width = image.shape[:2]
        canvas = np.zeros((height, width * 4, 3), dtype=np.uint8)
        
        # Resize reference image to match height if needed
        if reference_image.shape[0] != height:
            aspect_ratio = reference_image.shape[1] / reference_image.shape[0]
            new_width = int(height * aspect_ratio)
            reference_image_resized = cv2.resize(reference_image, (new_width, height))
        else:
            reference_image_resized = reference_image
        
        # Add original image to the canvas
        canvas[:, 0:width] = image
        
        # Add mask to the canvas (convert grayscale to RGB for display)
        mask_rgb = cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB)
        canvas[:, width:width*2] = mask_rgb
        
        # Add reference image to the canvas
        canvas[:, width*2:width*3] = reference_image_resized[:, :width]  # Crop if needed
        
        # Add inpainted result to the canvas
        canvas[:, width*3:width*4] = result
        
        # Add titles
        title_height = 30
        title_canvas = np.ones((title_height, width * 4, 3), dtype=np.uint8) * 255
        
        # Add text for titles
        cv2.putText(title_canvas, "Original Image", (width//2 - 70, 20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(title_canvas, "Mask", (width + width//2 - 30, 20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(title_canvas, "Reference Image", (width*2 + width//2 - 80, 20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(title_canvas, "Inpainted Result", (width*3 + width//2 - 80, 20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        
        # Combine title and images
        display_image = np.vstack((title_canvas, canvas))
        
        # Save the combined image
        cv2.imwrite(sample_output_path, cv2.cvtColor(display_image, cv2.COLOR_RGB2BGR))
        
        print(f"Image saved successfully to {sample_output_path}")
    else:
        print(f"Sample image not found at {sample_image_path}")
        print("Please update the path to test with your own image")

except ImportError as e:
    print(f"PaintByExample dependencies not installed: {e}")
    print("Please install iopaint package with: pip install iopaint")
    print(f"Full error: {str(e)}")
    import traceback
    traceback.print_exc()
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

2025-05-04 21:41:39.162 | INFO     | iopaint.model_manager:init_model:47 - Loading model: Fantasy-Studio/Paint-by-Example


True
Error: Unsupported model: Fantasy-Studio/Paint-by-Example. Available models: ['lama', 'ldm', 'cv2']


Traceback (most recent call last):
  File "C:\Users\rberg\AppData\Local\Temp\ipykernel_22164\1165173705.py", line 24, in <module>
    model = ModelManager(
  File "c:\Users\rberg\Anaconda3\envs\seg-env-win-friendly\lib\site-packages\iopaint\model_manager.py", line 40, in __init__
    self.model = self.init_model(name, device, **kwargs)
  File "c:\Users\rberg\Anaconda3\envs\seg-env-win-friendly\lib\site-packages\iopaint\model_manager.py", line 49, in init_model
    raise NotImplementedError(
NotImplementedError: Unsupported model: Fantasy-Studio/Paint-by-Example. Available models: ['lama', 'ldm', 'cv2']


In [1]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
print(f"FlashAttention available: {torch.backends.cuda.flash_sdp_enabled()}")

PyTorch version: 2.3.1
CUDA available: True
CUDA version: 12.1
FlashAttention available: True


In [7]:
import config
import os
import cv2

In [12]:
inpainted_dir = config.INPAINTED_DATASET_PATH
mask_dir = config.MASKS_DATASET_PATH
masked_dir = config.MASKED_DATASET_PATH
original_dir = config.ORIGINAL_DATASET_PATH

file_names = ['000000469300.png', '000000251082.png', '000000048340.png', '000000100726.png', '000000136790.png']

for i, file_name in enumerate(file_names):
    inpainted_path = os.path.join(inpainted_dir, file_name)
    mask_path = os.path.join(mask_dir, file_name)
    masked_path = os.path.join(masked_dir, file_name)
    original_path = os.path.join(original_dir, file_name)

    inpainted_image = cv2.imread(inpainted_path)
    mask_image = cv2.imread(mask_path)
    masked_image = cv2.imread(masked_path)
    original_image = cv2.imread(original_path)
    # Create output directory if it doesn't exist
    output_dir = os.path.join("./byo_gan_results", "report-images")
    os.makedirs(output_dir, exist_ok=True)
    
    # Get index from filename (assuming it's in the loop)
    i = file_names.index(file_name)
    
    # Save inpainted image
    inpainted_output_path = os.path.join(output_dir, f"inpainted_{i+1}.jpg")
    cv2.imwrite(inpainted_output_path, inpainted_image)
    
    # Save mask image
    mask_output_path = os.path.join(output_dir, f"mask_{i+1}.jpg")
    cv2.imwrite(mask_output_path, mask_image)

    masked_output_path = os.path.join(output_dir, f"masked_{i+1}.jpg")
    cv2.imwrite(masked_output_path, masked_image)
    
    original_output_path = os.path.join(output_dir, f"original_{i+1}.jpg")
    cv2.imwrite(original_output_path, original_image)

    print(f"Saved inpainted image to {inpainted_output_path}")
    print(f"Saved mask image to {mask_output_path}")
    print(f"Saved masked image to {masked_output_path}")
    print(f"Saved original image to {original_output_path}")
    



Saved inpainted image to ./byo_gan_results\report-images\inpainted_1.jpg
Saved mask image to ./byo_gan_results\report-images\mask_1.jpg
Saved masked image to ./byo_gan_results\report-images\masked_1.jpg
Saved inpainted image to ./byo_gan_results\report-images\inpainted_2.jpg
Saved mask image to ./byo_gan_results\report-images\mask_2.jpg
Saved masked image to ./byo_gan_results\report-images\masked_2.jpg
Saved inpainted image to ./byo_gan_results\report-images\inpainted_3.jpg
Saved mask image to ./byo_gan_results\report-images\mask_3.jpg
Saved masked image to ./byo_gan_results\report-images\masked_3.jpg
Saved inpainted image to ./byo_gan_results\report-images\inpainted_4.jpg
Saved mask image to ./byo_gan_results\report-images\mask_4.jpg
Saved masked image to ./byo_gan_results\report-images\masked_4.jpg
Saved inpainted image to ./byo_gan_results\report-images\inpainted_5.jpg
Saved mask image to ./byo_gan_results\report-images\mask_5.jpg
Saved masked image to ./byo_gan_results\report-image